Importing Libraries

In [239]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


This notebooks gives examples on how to use the BayesianOptimization class.
It has built in funcionality for the following:
1. Suggesting more than 1 candidate ($q>1$) per iteration.
2. 3 different types of GP models (Single Task GP, Mixed Single Task GP, SAASBO (Model List GP for $d_{out}>1$)). 
3. Multi-Objective and Single-Objective optimization.
4. Weights for prioritizing targets.
5. Ingredient minimization with weights.
6. 3 different types of acquisition functions for Single-Objective optimization (UCB, EI, PI).
7. Expected Hypervolume Improvement for Multi-Objective optimization.
8. 2 different $q$-sampling strategies (Believer Update and Monte-Carlo). 

In [240]:
from bayesian_opt import BayesianOptimization
import numpy as np
import pandas as pd

Example usage of bayesian optimization for continuous data, where $d_\text{out}=1$. 

In [241]:
train_X = np.random.uniform(low = 0, high = 10, size = (10,3))
df = pd.DataFrame(train_X, columns=['pH','c1','c2'])
df

,pH,c1,c2
0,8.643471,9.573514,0.770545
1,1.780722,5.510784,8.308036
2,7.793581,5.550843,5.902989
3,7.691560,4.169780,7.801505
4,6.128419,7.993692,8.407853
5,7.159595,5.869436,2.482776
6,2.858901,6.821077,3.498308
7,2.609621,5.304731,4.490708
8,1.561926,4.570062,5.639204
9,3.323434,6.557455,7.064134


In [242]:
bo_model = BayesianOptimization().fit(df, ['c2','c1'], model_type='Single-Task GP')

a = bo_model.candidates(1, export_df=True)
a

c:\Users\danin\AppData\Local\Programs\Python\Python313\Lib\site-packages\botorch\models\utils\assorted.py:264: InputDataWarning: Data (input features) is not contained to the unit cube. Please consider min-max scaling the input data.
  check_min_max_scaling(


,pH,c2,c1
0,6.128419,8.387966,7.981635


Example usage of bayesian optimization for mixed continuous and categorical data with target of dim 1 

In [243]:
train_X_cat = train_X
train_X_cat[:,0] = np.round(train_X_cat[:,0] )
df_cat = pd.DataFrame(train_X_cat, columns=['pH','c1','c2'])
df_cat


,pH,c1,c2
0,9.0,9.573514,0.770545
1,2.0,5.510784,8.308036
2,8.0,5.550843,5.902989
3,8.0,4.169780,7.801505
4,6.0,7.993692,8.407853
5,7.0,5.869436,2.482776
6,3.0,6.821077,3.498308
7,3.0,5.304731,4.490708
8,2.0,4.570062,5.639204
9,3.0,6.557455,7.064134


In [244]:
bo_model = BayesianOptimization().fit(df,['c1','c2'],cat_dims=['pH'], model_type='Single-Task GP')

a = bo_model.candidates(1, export_df=True)
a

c:\Users\danin\AppData\Local\Programs\Python\Python313\Lib\site-packages\botorch\models\utils\assorted.py:264: InputDataWarning: Data (input features) is not contained to the unit cube. Please consider min-max scaling the input data.
  check_min_max_scaling(


,c1,c2,pH_2.0,pH_3.0,pH_6.0,pH_7.0,pH_8.0,pH_9.0,c1,c2
0,7.993684,8.407844,1.923489e-07,0.000007,0.999993,0.0,0.0,1.135237e-42,7.981635,8.387968


Choosing a different GP model type.

In [245]:
bo_model = BayesianOptimization().fit(df,['c1'],cat_dims=['pH'], model_type='SAASBO')

a = bo_model.candidates(2, export_df=True)
a

c:\Users\danin\AppData\Local\Programs\Python\Python313\Lib\site-packages\botorch\models\utils\assorted.py:264: InputDataWarning: Data (input features) is not contained to the unit cube. Please consider min-max scaling the input data.
  check_min_max_scaling(
c:\Users\danin\AppData\Local\Programs\Python\Python313\Lib\site-packages\botorch\models\utils\assorted.py:267: InputDataWarning: Data (outcome observations) is not standardized (std = tensor([1.6226], dtype=torch.float64), mean = tensor([6.1921], dtype=torch.float64)).Please consider scaling the input to zero mean and unit variance.
  check_standardization(Y=train_Y, raise_on_fail=raise_on_fail)


,c1,c2,pH_2.0,pH_3.0,pH_6.0,pH_7.0,pH_8.0,pH_9.0,c1
0,9.573514,0.770545,0.0,1.0,1.0,0.0,1.0,1.0,9.53574
1,9.573514,8.407853,1.0,1.0,0.0,1.0,1.0,1.0,9.45992


Different $q$-sampling strategy.

In [246]:
bo_model = BayesianOptimization().fit(df,['c1'])

a = bo_model.candidates(4, q_sampling_method="Believer", export_df=True)
a

c:\Users\danin\AppData\Local\Programs\Python\Python313\Lib\site-packages\botorch\models\utils\assorted.py:264: InputDataWarning: Data (input features) is not contained to the unit cube. Please consider min-max scaling the input data.
  check_min_max_scaling(
c:\Users\danin\AppData\Local\Programs\Python\Python313\Lib\site-packages\botorch\models\utils\assorted.py:264: InputDataWarning: Data (input features) is not contained to the unit cube. Please consider min-max scaling the input data.
  check_min_max_scaling(
c:\Users\danin\AppData\Local\Programs\Python\Python313\Lib\site-packages\botorch\models\utils\assorted.py:264: InputDataWarning: Data (input features) is not contained to the unit cube. Please consider min-max scaling the input data.
  check_min_max_scaling(
c:\Users\danin\AppData\Local\Programs\Python\Python313\Lib\site-packages\botorch\models\utils\assorted.py:264: InputDataWarning: Data (input features) is not contained to the unit cube. Please consider min-max scaling the i

,pH,c2,c1
0,5.747257,8.146573,7.006119
1,9.000000,0.770545,9.551378
2,6.396983,8.407853,7.078368
3,9.000000,0.770545,9.551422


Choosing a different acquisition function.

In [252]:
a = bo_model.candidates(4, export_df=True, acq_func_name="UCB")

Minimizing Ingredients

In [248]:
a = bo_model.candidates(4, q_sampling_method="Believer", export_df=True, input_weights={0:1.1, 1:2.5, 2:2.0})
a

c:\Users\danin\AppData\Local\Programs\Python\Python313\Lib\site-packages\botorch\models\utils\assorted.py:264: InputDataWarning: Data (input features) is not contained to the unit cube. Please consider min-max scaling the input data.
  check_min_max_scaling(
c:\Users\danin\AppData\Local\Programs\Python\Python313\Lib\site-packages\botorch\models\utils\assorted.py:264: InputDataWarning: Data (input features) is not contained to the unit cube. Please consider min-max scaling the input data.
  check_min_max_scaling(
c:\Users\danin\AppData\Local\Programs\Python\Python313\Lib\site-packages\botorch\models\utils\assorted.py:264: InputDataWarning: Data (input features) is not contained to the unit cube. Please consider min-max scaling the input data.
  check_min_max_scaling(


,pH,c2,c1
0,3.105260,2.936671,6.350360
1,2.471435,3.323701,6.303975
2,8.935767,0.902177,9.148961
3,5.615184,8.336496,7.071876


Multi-Objective optimization.

In [253]:
bo_model = BayesianOptimization().fit(df,['c1', 'c2'])

a = bo_model.candidates(4, export_df=True)
a

c:\Users\danin\AppData\Local\Programs\Python\Python313\Lib\site-packages\botorch\models\utils\assorted.py:264: InputDataWarning: Data (input features) is not contained to the unit cube. Please consider min-max scaling the input data.
  check_min_max_scaling(
c:\Users\danin\AppData\Local\Programs\Python\Python313\Lib\site-packages\botorch\acquisition\multi_objective\monte_carlo.py:111: NumericsWarning: qExpectedHypervolumeImprovement has known numerical issues that lead to suboptimal optimization performance. It is strongly recommended to simply replace

	 qExpectedHypervolumeImprovement 	 --> 	 qLogExpectedHypervolumeImprovement 

instead, which fixes the issues and has the same API. See https://arxiv.org/abs/2310.20708 for details.
  legacy_ei_numerics_warning(legacy_name=type(self).__name__)


,pH,c1,c2
0,6.009131,7.979856,8.385029
1,3.548824,6.193117,5.425084
2,6.055599,7.916877,8.281121
3,5.958598,7.945436,8.328275
